# Содержание
- [Настройка консольной утилиты yc и библиотеки yandex_neurosupport](#yc-setting)
- [Готовим документацию и создаем индекс](#cook-index)
- [Получаем ответы по документации](#take-my-money)
- [Смотрим документы в индексе](#look-for-docs)
- [Добавляем новый документ в индекс](#add-doc)
- [Переключаемся между версиями индексов](#switch-index)
- [Удаляем документ из индекса](#delete-doc)
- [Удаляем весь индекс](#delete-all)

<a id="yc-setting"></a>
## Настройка консольной утилиты yc
Все команды выполняются в терминале

In [1]:
# curl -sSL https://storage.yandexcloud.net/yandexcloud-yc/install.sh | bash #скачиваем
# yc init
# yc config list #test
# yc iam key create --output yndx_cloud_key.json --service-account-name user-1 # задайте свои имена 'yndx_cloud_key.json', 'user-1'
# yc config profile create service_account_profile
# yc config set service-account-key yndx_cloud_key.json     # задайте свое имя 'yndx_cloud_key.json'
# yc iam create-token

# формируем тестовый запрос
# IAM_TOKEN=$(yc iam create-token)
# FOLDER_ID="${1:-<your_folder_id>}"
# curl -H "Authorization: Bearer ${IAM_TOKEN}" -H "x-folder-id: ${FOLDER_ID}" -X GET https://supportgpt.api.cloud.yandex.net/indexer/v1/indexes

Устанавливаем библиотеку

In [2]:
# !pip install yandex_neurosupport    # устанавливаем билиотеку
# !pip show yandex_neurosupport       # проверяем установку

In [3]:
import yandex_neurosupport as nrsprt

In [4]:
# импорт других библиотек, которые нам понадобятся
import os
import requests
import markdownify              # при необходимости устанавливаем через pip install markdownify, чтобы конвертировать html->markdown
from dotenv import load_dotenv  # при необходимости устанавливаем через pip install python-dotenv, чтобы использовать переменные из .env
load_dotenv()

True

Создаем клиента

In [5]:
client = nrsprt.YandexCloudNeuroSupportClient(
    auth_token=nrsprt.get_iam_token(),  # Или используйте свой способ
    folder_id=os.getenv('FOLDER_ID'),   # Или укажите вручную: 'your_folder_id'
    service=os.getenv('SERVICE'),       # Или укажите вручную: 'your_service'
    product=os.getenv('PRODUCT')        # Или укажите вручную: 'your_product'
)

Проверяем доступность api

In [6]:
client.check_api()

True

<a id="cook-index"></a>
## Готовим документацию и создаем индекс

Скачиваем документацию Нейросаппорта (https://yandex.cloud/ru/docs/neurosupport/quick-start).

In [7]:
def get_html_content(url: str) -> str | None:
    """
    Получает HTML содержимое веб-страницы по указанному URL.

    Args:
        url: URL веб-страницы для загрузки

    Returns:
        HTML содержимое страницы в виде строки или None в случае ошибки
    """
    try:
        response: requests.Response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при загрузке страницы: {e}")
        return None

# cкачиваем документацию по нейросаппорту
url = "https://yandex.cloud/ru/docs/neurosupport/quick-start"
html_content = get_html_content(url)
markdown_page = markdownify.markdownify(html_content, heading_style="ATX")
markdown_page_clean = '# Нейросаппорт'+markdown_page.split('# Нейросаппорт')[1] #удаляем неинформативную шапку
print(markdown_page_clean[:300])

# Нейросаппорт

Статья создана

[![](https://storage.yandexcloud.net/cloud-www-assets/constructor/content-program/icons/yandexcloud.svg)](/ru)

[Yandex Cloud](/ru)

Обновлена 2 августа 2025 г.

* [О сервисе](#about)
* [Получение доступа](#get-access)
* [Интеграция](#integration)
  + [Архитектура инт


In [8]:
# Создаем документ согласно инструкции к API
documents = [
    {
        "doc_id": "doc_1",
        "title": "Инструкция Нейросаппорта",
        "text": markdown_page_clean,
        "meta": {
                "title": "Дока нейросаппорта",
                "source_id": "https://yandex.cloud/ru/docs/neurosupport/quick-start",
                "category": "quick-start"
            }
        }
]

Создаем индекс c именем `INDEX_NAME`.

In [9]:
PREFIX_INDEX = os.getenv('PREFIX_INDEX') # PREFIX_INDEX - выдается после обработки заявки на подключение
INDEX_NAME = PREFIX_INDEX+'neurosupport_index'

In [10]:
# Создаем индекс
res = client.create_or_update_index(
    index_name=INDEX_NAME ,
    documents=documents,
    meta={"description": "Инструкция Нейросаппорта"},
    auto_switch=True,
    diff=False
)

nrsprt.mask_response_fields(res['body'], values_to_mask=[PREFIX_INDEX])

{'index_name': '******neurosupport_index',
 'index_version': '1',
 'index_parent_version': None}

In [11]:
# Как получить request-id данного запроса (особенно актуально при обращении в поддержку)
res['headers']['x-request-id']

'85f3efad-a919-4bf6-9aba-933ba89c5066'

<a id="take-my-money"></a>
## Получаем ответы по документации

In [12]:
# Пример диалога
sample_dialog = [
    {
        "role": "operator",
        "text": "Добрый день, чем я могу помочь?"
    },
    {
        "role": "client",
        "text": "Какая система оценок ответов, простыми словами"
    }
]

# Получаем ответ
answers = client.get_generative_answer(index_name=INDEX_NAME, dialog=sample_dialog)
print(answers['body']['answers'][0]['answer'])

Система оценок ответов выглядит так:

**5 — Идеально.** Ответили на все вопросы и дали отличный безошибочный ответ пользователю. 

**4 — Хорошо.** Написанный текст решает вопрос пользователя и по алгоритму является верным, но есть небольшие ошибки в формулировках и/или небольшие расхождения с редакционной политикой. 

**3 — Средне.** В целом написанный текст отвечает на обращение, соответствует базе знаний и приближает к решению задачи, но требует правок (фразы и предложения): не дописали какую-то значимую информацию, ответили не на все вопросы пользователя, если их было несколько. 

**2 — Плохо.** Ответ абсолютно бесполезен и его нельзя было использовать в диалоге. Нужно писать правильный ответ с нуля. Совершенно не соответствует базе знаний. Не соответствует действительности и входным данным (например, решили проблему не по тому обращению или ответили не на тот вопрос). Дезинформирует пользователя. 

**1 — Критично.** Очень плохой ответ. Может вызвать риски, если пользователь получит

<a id="look-for-docs"></a>
## Смотрим документы в индексе

Внимание! Если загружаемы документ был большого размера, то он перед индексирование разбивается на более мелкие куски(чанки) и к id документа добавляетя суффикс `_{номер чанка}`

In [13]:
client.get_documents_from_index(index_name=INDEX_NAME, document_id='doc_1_7')

{'body': {'parameters': {'items_total': 1,
   'items_returned': 1,
   'last_id': 'doc_1_7',
   'after_id': None,
   'limit': 100,
   'sort_by': 'document_id',
   'sort_order': 'asc'},
  'items': [{'doc_id': 'doc_1_7',
    'title': 'Инструкция Нейросаппорта',
    'text': '```\n\n### [Рекомендации к тестированию и оценке качества](#qa-recommendations)Рекомендации к тестированию и оценке качества\n\nДля тестирования рекомендуется подготовить релевантный набор данных, соответствующий потоку, на котором в дальнейшем планируется использовать сервис. Например, если планируется запускать сервис только для 1-й линии поддержки, то в тестировании должны присутствовать данные только с нее. Либо, если планируется запускать сервис по определенным тематикам, то набор данных должен отражать реальное распределение по тематикам. Это необходимо, чтобы выводы, сделанные на тестовом наборе данных, можно было потом корректно обобщить.\n\nРекомендуется выбирать ту часть потока обращений, для которой:\n\n* ба

<a id="add-doc"></a>
## Добавляем новый документ в индекс

Зададим вопрос, ответа на который нет в докумментации.

In [14]:
sample_dialog_2 = [
    {
        "role": "operator",
        "text": "Добрый день, чем я могу помочь?"
    },
    {
        "role": "client",
        "text": "Как получить список доступных сервисных аккаунтов в каталоге по умолчанию?"
    }
]

# Получаем ответ не очень рельвантный запросу, т.к. у нас нет инструкции конкретно про IAM токен
answers = client.get_generative_answer(index_name=INDEX_NAME, dialog=sample_dialog_2)
print(answers['body']['answers'][0]['answer'])

В предоставленных текстах нет ответа на ваш вопрос. Однако, если у вас возникнут другие вопросы, связанные с использованием Нейросаппорта или вам понадобится дополнительная информация по другим темам, пожалуйста, обращайтесь! Если вопрос касается работы с другими сервисами, рекомендуем обратиться к соответствующей документации или поддержке.


Скачаем и добавим подходящий документ (версия индекса увеличивает на +1)

In [15]:
# cкачиваем инструкци по IAM
url = "https://yandex.cloud/ru/docs/iam/operations/iam-token/create-for-sa"
html_content = get_html_content(url)
markdown_page = markdownify.markdownify(html_content, heading_style="ATX")

new_documents = [
    {
        "doc_id": "doc_2",
        "title": "Получение IAM-токена для сервисного аккаунта",
        "text": markdown_page,
        "meta": {
                "title": "Дока yandex cloud",
                "source_id": url,
                "category": "instruction"
                }
        }
    ]

Смотрим кол-во документов в индексе до добавления

In [16]:
res = client.get_index_info(index_name=INDEX_NAME)
nrsprt.mask_response_fields(res['body'], values_to_mask=[PREFIX_INDEX])

{'index_name': '******neurosupport_index',
 'index_version': 1,
 'index_parent_version': None,
 'doc_count': 8}

Добавляем

In [17]:
res = client.create_or_update_index(
    index_name=INDEX_NAME,
    documents=new_documents,
    meta={"description": "Инструкция Нейросаппорта и Облака"},
    auto_switch=True,
    diff=True
)

In [18]:
nrsprt.mask_response_fields(res['body'], values_to_mask=[PREFIX_INDEX])

{'index_name': '******neurosupport_index',
 'index_version': '2',
 'index_parent_version': '1'}

Смотрим кол-во документов в новом индексе после добавления

In [19]:
res = client.get_index_info(index_name=INDEX_NAME)
nrsprt.mask_response_fields(res['body'], values_to_mask=[PREFIX_INDEX])

{'index_name': '******neurosupport_index',
 'index_version': 2,
 'index_parent_version': 1,
 'doc_count': 16}

Смотрим как модель отвечает с учетом нового рельвантного документа

In [20]:
# Получаем ответ отличный ответ, т.к. у нас есть  инструкции конкретно про IAM токен
answers = client.get_generative_answer(index_name=INDEX_NAME, dialog=sample_dialog_2)
print(answers['body']['answers'][0]['answer'])


Получите список доступных сервисных аккаунтов в каталоге по умолчанию с помощью команды: 

```
yc iam service-accounts list
```


<a id="switch-index"></a>
## Переключаемся между версиями индексов

Текущая версия

In [21]:
res = client.get_index_info(index_name=INDEX_NAME)
nrsprt.mask_response_fields(res['body'], values_to_mask=[PREFIX_INDEX])

{'index_name': '******neurosupport_index',
 'index_version': 2,
 'index_parent_version': 1,
 'doc_count': 16}

Переключимся на предыдущую

In [22]:
res = client.switch_index_version(index_name=INDEX_NAME, index_version=1)
nrsprt.mask_response_fields(res['body'], values_to_mask=[PREFIX_INDEX])

{'index_name': '******neurosupport_index', 'index_version': 1}

Проверяем текущую версию

In [23]:
res = client.get_index_info(index_name=INDEX_NAME)
nrsprt.mask_response_fields(res['body'], values_to_mask=[PREFIX_INDEX])

{'index_name': '******neurosupport_index',
 'index_version': 1,
 'index_parent_version': None,
 'doc_count': 8}

In [24]:
# версия с которой переключились
res = client.get_index_info(index_name=INDEX_NAME, index_version=2)
nrsprt.mask_response_fields(res['body'], values_to_mask=[PREFIX_INDEX])

{'index_name': '******neurosupport_index',
 'index_version': 2,
 'index_parent_version': 1,
 'doc_count': 16}

Вернемся обратно

In [25]:
res = client.switch_index_version(index_name=INDEX_NAME, index_version=2)
nrsprt.mask_response_fields(res['body'], values_to_mask=[PREFIX_INDEX])

{'index_name': '******neurosupport_index', 'index_version': 2}

<a id="delete-doc"></a>
## Удаление документов из индекса.

In [26]:
# cкачиваем нашу статью с хабр
url = "https://habr.com/ru/companies/yandex/articles/908972/"
html_content = get_html_content(url)
markdown_page = markdownify.markdownify(html_content, heading_style="ATX")

new_documents_3 = [
    {
        "doc_id": "doc_3",
        "title": "Статья",
        "text": markdown_page,
        "meta": {
                "title": "Статья на хабре",
                "source_id": url,
                "category": "article"
                }
        }
    ]

#добавляем в новый индекс
client.create_or_update_index(
        index_name=INDEX_NAME,
        documents=new_documents_3,
        meta={"description": "Инструкция Нейросаппорта и Облака и Хабр-статья"},
        auto_switch=True,
        diff=True
)

res = client.get_index_info(index_name=INDEX_NAME)
nrsprt.mask_response_fields(res['body'], values_to_mask=[PREFIX_INDEX])

{'index_name': '******neurosupport_index',
 'index_version': 3,
 'index_parent_version': 2,
 'doc_count': 24}

Задаим специфический вопрос к новому контенту

In [27]:
# Пример диалога
sample_dialog_3 = [
    {
        "role": "operator",
        "text": "Добрый день, чем я могу помочь?"
    },
    {
        "role": "client",
        "text": "Как собирать данные для DPO?"
    }
]

# Получаем ответ
answers = client.get_generative_answer(index_name=INDEX_NAME, dialog=sample_dialog_3)
print(answers['body']['answers'][0]['answer'])

Для стадии DPO мы не собирали разметку редакторами, а использовали синтетический DPO, в котором выбор позитивного и негативного ответа происходит без участия человека. Мы пришли к следующей схеме сбора данных для DPO:

* берём тот же набор инстрактов, что и для SFT-стадии;
* генерируем ответы-кандидаты набором лучших моделей;
* позитивный и негативный ответ отбираем по ансамблю ревордов: базовые реворд-модели YandexGPT, набор специализированных для Yandex Neurosupport ревордов, обученных на разных подмножествах данных;
* дополнительно на позитивный ответ добавляем ограничение по подтверждённости с golden-ответом из SFT.

 


In [28]:
# наш загружаемый документ разбился на 6 чанков (можете самостоятельно убедиться)
# удалим последнюю статью, передав список документов на удаление
res = client.delete_documents_from_index(
    index_name=INDEX_NAME,
    docs_ids=['doc_3_0', 'doc_3_1', 'doc_3_2', 'doc_3_3', 'doc_3_4', 'doc_3_5', 'doc_3_6', 'doc_3_7']
)
nrsprt.mask_response_fields(res['body'], values_to_mask=[PREFIX_INDEX])

{'index_name': '******neurosupport_index',
 'index_version': '4',
 'index_parent_version': '3'}

In [29]:
res = client.get_index_info(index_name=INDEX_NAME)
nrsprt.mask_response_fields(res['body'], values_to_mask=[PREFIX_INDEX])

{'index_name': '******neurosupport_index',
 'index_version': 4,
 'index_parent_version': 3,
 'doc_count': 16}

Зададим тотже вопрос

In [30]:
# т.к. необходимые для ответа данные удалились, то не получим ответ на заданный вопрос
answers = client.get_generative_answer(index_name=INDEX_NAME, dialog=sample_dialog_3)
print(answers['body']['answers'][0]['answer'])

Пожалуйста, уточните запрос — в предоставленных материалах нет информации по вашему вопросу. Возможно, вы имели в виду что-то конкретное, но для более точного ответа требуется дополнительная информация. Если у вас есть другие вопросы или нужна помощь в рамках работы с нашим сервисом, пожалуйста, сформулируйте вопрос иначе или предоставьте больше деталей. 

Для получения помощи по другим вопросам, пожалуйста, обращайтесь!


Попробуем найти по индексу

In [31]:
#пустой результат 'items': []
client.get_documents_from_index(index_name=INDEX_NAME, document_id="doc_3_0", sort_order="desc")


{'body': {'parameters': {'items_total': 0,
   'items_returned': 0,
   'last_id': None,
   'after_id': None,
   'limit': 100,
   'sort_by': 'document_id',
   'sort_order': 'desc'},
  'items': []},
 'headers': {'content-length': '149',
  'content-type': 'application/json',
  'date': 'Wed, 06 Aug 2025 13:49:45 GMT',
  'x-request-id': '34f7b5a6-be00-4a3c-8d57-3e5b17fa00df',
  'server': 'ycalb'}}

<a id="delete-all"></a>
## Удаление индекса. Внимание!!! Удаляется весь индекс полностью со всеми версиями.

Посмотрим на все доступные индексы

In [32]:
res = client.get_indexes_full()
nrsprt.mask_response_fields(res['body'], values_to_mask=[PREFIX_INDEX])

{'indexes': [{'index_id': 45,
   'name': '******neurosupport_index',
   'current_version': 4,
   'doc_count': 16}],
 'indexes_count': 31,
 'pages_count': 1}

Удалим наш индекс `INDEX_NAME`

In [33]:
client.delete_index(index_name=INDEX_NAME)

{'body': None,
 'headers': {'content-length': '4',
  'content-type': 'application/json',
  'date': 'Wed, 06 Aug 2025 13:49:51 GMT',
  'x-request-id': '9980be8e-903a-4162-88f0-eb740e67ce8b',
  'server': 'ycalb'}}

Снова смотрим на все доступные индексы

In [34]:
res = client.get_indexes_full()
nrsprt.mask_response_fields(res['body'], values_to_mask=[PREFIX_INDEX])

{'indexes': [], 'indexes_count': 31, 'pages_count': 1}